In [1]:
from handnormalizer import normalize_text
from handnormalizer.parser.pokerstars_parser import PokerStarsParser

parser = PokerStarsParser(normalize_text)
parser.process_folder("../dataset", "output")


In [1]:
import handnormalizer.parser.pokerstars_parser as p
p.__file__



'/home/noarones/GitPr/poker-hand-normalizer/src/handnormalizer/parser/pokerstars_parser.py'

PokerStars Hand #257298513408:  Hold'em No Limit ($0.05/$0.10) - 2025/08/11 9:33:56 CET [08/11/2025 03:33:56 ET]
Table 'Caph' 6-max Seat #1 is the button
Seat 1: Player1 ($10.04 in chips)
Seat 2: Player2 ($16.58 in chips)
Seat 3: Hero ($8 in chips)
Seat 4: Player3 ($3.69 in chips)
Seat 5: Player4 ($46.79 in chips)
Seat 6: Player5 ($10 in chips)
Player2: posts small blind $0.05
Hero: posts big blind $0.10
*** HOLE CARDS ***
Dealt to Hero [8d 8s]
Player3: raises $0.20 to $0.30
Player4: folds
Player5: folds
Player1: raises $0.60 to $0.90
Player2: folds
Hero: folds
Player3: calls $0.60
*** FLOP *** [Kd Ad Kc]
Player3: checks
Player1: checks
*** TURN *** [Kd Ad Kc] [5c]
Player3: checks
Player1: bets $0.61
Player3: folds
Uncalled bet ($0.61) returned to Player1
Player1 collected $1.85 from pot
Player1: doesn't show hand
*** SUMMARY ***
Total pot $1.95 | Rake $0.10
Board [Kd Ad Kc 5c]
Seat 1: Player1 (button) collected ($1.85)
Seat 2: Player2 (small blind) folded before Flop
Seat 3: Hero (big